In [38]:
%pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
import pdfplumber



In [41]:
pdf_path = "JBCAP.pdf"

In [73]:
# Read tables from a PDF file
pf = pdfplumber.open(pdf_path)

In [74]:
pgs = pf.pages
data = []

In [75]:
for pg in range(79,140):
    z = pf.pages[pg].extract_table(table_settings={"vertical_strategy": "text"})
    if z:
        data.extend(z)

In [69]:
print(z)

[['', 'Sale Detai', 'ls', '', '', '', 'Purchase Deta', 'ils', '', '', '', 'Gain / Loss', ''], ['Sell Date', 'Quantity', 'Sell Rate', 'Total Sale\nValue', 'Purchase\nDate', 'Purchase Stamp\nRate Duty', 'Actual Cost', 'FMV as on\n31-01-2018/\nIndexed Rate', 'Applicable\nE\nRate', 'ffective Cost', 'Days\nHeld', 'Short Term Long Term', 'Effective'], ['Equity', '', '', '', '', '', '', '', '', '', '', '', ''], ['AADHAR HOUSIN', 'G FINANCE L', 'TD', '', '', '', '', '', '', '', '', '', ''], ['23-Sep-2024', '5,000', '497.744', '24,88,720.40\n13,03,094.00\n1,36,19,664.74\n12,61,080.07\n12,61,080.07\n12,61,080.07\n12,61,080.07\n15,13,296.08\n12,01,557.09\n2,51,70,652.59', '12-Jun-2024', '387.09', '19,35,464.50', '', '387.09', '19,35,464.50', '103', '5,53,256.11', ''], ['23-Sep-2024', '2,618', '497.744', None, '12-Jun-2024', '387.09', '10,13,409.21', '', '387.09', '10,13,409.21', '103', '2,89,684.90', ''], ['24-Sep-2024', '27,000', '504.432', None, '12-Jun-2024', '387.09', '1,04,51,508.30', '', '3

In [70]:
import pandas
import numpy as np
df = pandas.DataFrame(np.array(z))